# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [144]:
# Libraries
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [145]:
# your code here
admissions=pd.read_csv('data\Admission_Predict.csv')
admissions

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Remove trailing spaces at the end of the column names if there are any.


In [146]:
# your code here
admissions.columns = admissions.columns.str.rstrip()
admissions

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Let's evaluate the dataset by looking at the `head` function.

In [147]:
# your code here
admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [148]:
# your code here
admissions.isna().any() #NO MISSING DATA
#admissions[admissions.columns[admissions.isna().any()]]

Serial No.           False
GRE Score            False
TOEFL Score          False
University Rating    False
SOP                  False
LOR                  False
CGPA                 False
Research             False
Chance of Admit      False
dtype: bool

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [149]:
# your code here
admissions.set_index('Serial No.', inplace=True)
admissions

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84
383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [150]:
# your code here
admissions.reset_index(inplace=True)
admissions

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [151]:
admissions.set_index(['Serial No.','GRE Score', 'TOEFL Score'], inplace=True)

In [152]:
admissions

,,,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,GRE Score,TOEFL Score,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84
383,330,116,4,5.0,4.5,9.45,1,0.91


Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [153]:
# your code here
admissions.reset_index(inplace=True)

In [154]:
admissions

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [155]:
# your code here
admissions[(admissions['CGPA']>9)&(admissions['Research']==1)].count()
#101 studients with GPA is higher than 9 and have conducted research


Serial No.           101
GRE Score            101
TOEFL Score          101
University Rating    101
SOP                  101
LOR                  101
CGPA                 101
Research             101
Chance of Admit      101
dtype: int64

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [156]:
# your code here
admissions[(admissions['CGPA']>9)&(admissions['SOP']<3.5)]['Chance of Admit'].mean()

#The mean chance of admit for these applicants is 0.8019999999999999

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [157]:
#THEORY

"""
DataFrame.apply(self, func, axis=0)
Axis along which the function is applied:

0 or ‘index’: apply function to each COLUMN.  #by defect
1 or ‘columns’: apply function to each ROW.
"""

'\nDataFrame.apply(self, func, axis=0)\nAxis along which the function is applied:\n\n0 or ‘index’: apply function to each COLUMN.  #by defect\n1 or ‘columns’: apply function to each ROW.\n'

In [158]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    # your code here
    #I don't like too much passing an string as argument.
    std_column=(admissions[col]- np.mean(admissions[col]))/np.std(admissions[col])
    return std_column
    
#Version 2: passing column mean and column std as arguments:
############################################################
def standardize2(col,col_mean,col_std):
    return (col-col_mean)/col_std

"""
def standardize3(col):
    #II'S NOT POSSIBLE TO ACCESS THE WHOLE COLUMN AT THIS POINT, SO THIS ONE WILL NOT WORK
    col_mean=np.mean(admissions[col])
    col_std=np.std(admissions[col])
    return (col-col_mean)/col_std
"""



"\ndef standardize3(col):\n    #II'S NOT POSSIBLE TO ACCESS THE WHOLE COLUMN AT THIS POINT, SO THIS ONE WILL NOT WORK\n    col_mean=np.mean(admissions[col])\n    col_std=np.std(admissions[col])\n    return (col-col_mean)/col_std\n"

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [159]:
# your code here

#Version 1: passing string as argument (I personally don't like this too much):
#######################################
admissions['CGPA_std']= standardize('CGPA')
admissions['GRE_std']= standardize('GRE Score') 
admissions['LOR_std']= standardize('LOR')


#Version 2: passing column as argument and mean and std as additional arguments (args=):
########################################################################################
##For one column:
admissions['CGPA_std2']=admissions['CGPA'].apply(standardize2,args=(
                                                                    np.mean(admissions['CGPA']),
                                                                    np.std(admissions['CGPA'])))
##Loop to apply to multiple columns:
for col_name in ['CGPA', 'GRE Score', 'LOR']:
    admissions[col_name+'_std2']=admissions[col_name].apply(standardize2,
                                                           args=(np.mean(admissions[col_name]),
                                                                 np.std(admissions[col_name])
                                                                )
                                                          )
#Rename GRE Score_std2 column:
admissions.rename(columns = {'GRE Score_std2':'GRE_std2'}, inplace = True)
admissions

#Version 3: This doesn't work!
##############################
#admissions['CGPA_std3']= admissions['CGPA'].apply(standardize3)




,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,CGPA_std2,GRE_std2,LOR_std2
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.750174,1.755663,1.193197
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,-0.992501,-0.063450,0.076840
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,0.121191,0.456297,-1.039517
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433,-0.236698,-0.481338
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,1.234884,1.149292,-0.481338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840,0.736216,0.629546,0.076840
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840,0.852571,0.716170,0.076840
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197,1.417729,1.149292,1.193197
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019,0.304036,-0.409947,0.635019


In [160]:
#TEST WITH LAMBA EXPRESIONS:
############################
#Another way of doing it, but it doesn't create new columns.
#https://www.delftstack.com/es/howto/python-pandas/pandas-normalize/#:~:text=La%20normalizaci%C3%B3n%20de%20las%20columnas%20implicar%C3%A1%20llevar%20los,pueden%20ser%20normalizadas%20por%20una%20variedad%20de%20funciones.
admisitions_test =  admissions
#APPLY LAMBDA EXPRESION TO ONE COLUM:
#####################################
admisitions_test.apply(lambda x: (x-x.mean())/ x.std() if x.name=='CGPA' else x, axis=0)

#APPLY LAMBDA EXPRESION TO MULTIBPLE COLUMS:
############################################
admisitions_test.apply(lambda x: (x-x.mean())/x.std() if x.name in ['CGPA','GRE Score','LOR'] else x,axis=0)



,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,CGPA_std2,GRE_std2,LOR_std2
0,1,1.753381,118,4,4.5,1.191647,1.747900,1,0.92,1.750174,1.755663,1.193197,1.750174,1.755663,1.193197
1,2,-0.063367,104,3,3.0,0.076740,-0.991211,1,0.72,-0.992501,-0.063450,0.076840,-0.992501,-0.063450,0.076840
2,3,0.455704,110,3,3.5,-1.038166,0.121034,1,0.80,0.121191,0.456297,-1.039517,0.121191,0.456297,-1.039517
3,4,-0.236391,103,2,2.0,-0.480713,-0.642597,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433,-0.236698,-0.481338
4,5,1.147798,115,5,4.5,-0.480713,1.233279,1,0.90,1.234884,1.149292,-0.481338,1.234884,1.149292,-0.481338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,0.628727,110,3,3.5,0.076740,0.735259,1,0.82,0.736216,0.629546,0.076840,0.736216,0.629546,0.076840
381,382,0.715239,107,3,3.0,0.076740,0.851463,1,0.84,0.852571,0.716170,0.076840,0.852571,0.716170,0.076840
382,383,1.147798,116,4,5.0,1.191647,1.415886,1,0.91,1.417729,1.149292,1.193197,1.417729,1.149292,1.193197
383,384,-0.409414,103,3,3.5,0.634193,0.303641,0,0.67,0.304036,-0.409947,0.635019,0.304036,-0.409947,0.635019


We will generate the decision choice at random using the code below. Please run the cell.

In [161]:
# Libraries
from random import choices

In [162]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [163]:
# your code 
decision_choice



['LOR_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'CGPA_s

In [164]:
#LOOKUP fucntion: https://www.w3resource.com/pandas/dataframe/dataframe-lookup.php 
#df['value'] = df.lookup(df.index, df['best'])
admissions['deciding_column'] = admissions.lookup(admissions.index, decision_choice)

#The 'lookup' method is deprecated and will beremoved in a future version.
#You can use DataFrame.melt and DataFrame.locas a substitute.

C:\Users\patri\AppData\Local\Temp/ipykernel_11216/3179937112.py:3: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  admissions['deciding_column'] = admissions.lookup(admissions.index, decision_choice)


In [166]:
#Checking result:
admissions['decision_choice'] = decision_choice
admissions


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,CGPA_std2,GRE_std2,LOR_std2,deciding_column,decision_choice
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.750174,1.755663,1.193197,1.193197,LOR_std
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,-0.992501,-0.063450,0.076840,-0.063450,GRE_std
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,0.121191,0.456297,-1.039517,0.121191,CGPA_std
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433,-0.236698,-0.481338,-0.481338,LOR_std
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,1.234884,1.149292,-0.481338,1.149292,GRE_std
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840,0.736216,0.629546,0.076840,0.736216,CGPA_std
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840,0.852571,0.716170,0.076840,0.852571,CGPA_std
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197,1.417729,1.149292,1.193197,1.193197,LOR_std
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019,0.304036,-0.409947,0.635019,0.304036,CGPA_std


In [167]:
#Drop column:
admissions = admissions.drop('decision_choice',axis=1)
admissions

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,CGPA_std2,GRE_std2,LOR_std2,deciding_column
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.750174,1.755663,1.193197,1.193197
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,-0.992501,-0.063450,0.076840,-0.063450
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,0.121191,0.456297,-1.039517,0.121191
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433,-0.236698,-0.481338,-0.481338
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,1.234884,1.149292,-0.481338,1.149292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840,0.736216,0.629546,0.076840,0.736216
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840,0.852571,0.716170,0.076840,0.852571
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197,1.417729,1.149292,1.193197,1.193197
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019,0.304036,-0.409947,0.635019,0.304036


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [180]:
# your code here
##I did it in 3 different ways:

#using APPLY() and LAMBA EXPRESSION:
####################################
admissions['decision1'] = admissions['deciding_column'].apply(lambda x: 1 if x>0.8 else 0)

#Another way of doing it. WHERE() METHOD:
#########################################
##np.where(condition, True, False)
admissions['decision2'] = np.where(admissions['deciding_column']>0.8, 1, 0)

##Create column using list COMPRENHENSION:
#########################################
admissions['decision3'] =[1 if a>0.8 else 0 for a in admissions['deciding_column']]

admissions

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,CGPA_std2,GRE_std2,LOR_std2,deciding_column,decision1,decision2,decision3
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.750174,1.755663,1.193197,1.193197,1,1,1
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,-0.992501,-0.063450,0.076840,-0.063450,0,0,0
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,0.121191,0.456297,-1.039517,0.121191,0,0,0
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433,-0.236698,-0.481338,-0.481338,0,0,0
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,1.234884,1.149292,-0.481338,1.149292,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840,0.736216,0.629546,0.076840,0.736216,0,0,0
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840,0.852571,0.716170,0.076840,0.852571,1,1,1
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197,1.417729,1.149292,1.193197,1.193197,1,1,1
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019,0.304036,-0.409947,0.635019,0.304036,0,0,0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [187]:
# your code here
admissions[admissions['decision1']==1].shape[0]
##Solution: 85 students

85

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [192]:
# your code here
adm_columns = list(admissions.columns)
adm_columns

['Serial No.',
 'GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR',
 'CGPA',
 'Research',
 'Chance of Admit',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std2',
 'GRE_std2',
 'LOR_std2',
 'deciding_column',
 'decision1',
 'decision2',
 'decision3']

In [197]:
admissions.columns = admissions.columns.str.replace(' ','_') #Remove spaces
admissions.columns = admissions.columns.str.replace('.','') #Remove spaces
admissions.columns = admissions.columns.str.lower() # lowering strings\n",
#admissions.columns = admissions.columns.str.replace(\".\",\"\")\n",
list(admissions.columns)

#I'm not removing numbers because I want them there.

C:\Users\patri\AppData\Local\Temp/ipykernel_11216/1701015007.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  admissions.columns = admissions.columns.str.replace('.','') #Remove spaces


['serial_no',
 'gre_score',
 'toefl_score',
 'university_rating',
 'sop',
 'lor',
 'cgpa',
 'research',
 'chance_of_admit',
 'cgpa_std',
 'gre_std',
 'lor_std',
 'cgpa_std2',
 'gre_std2',
 'lor_std2',
 'deciding_column',
 'decision1',
 'decision2',
 'decision3']

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [198]:
# your code here
admissions

,serial_no,gre_score,toefl_score,university_rating,sop,lor,cgpa,research,chance_of_admit,cgpa_std,gre_std,lor_std,cgpa_std2,gre_std2,lor_std2,deciding_column,decision1,decision2,decision3
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.750174,1.755663,1.193197,1.193197,1,1,1
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,-0.992501,-0.063450,0.076840,-0.063450,0,0,0
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,0.121191,0.456297,-1.039517,0.121191,0,0,0
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433,-0.236698,-0.481338,-0.481338,0,0,0
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,1.234884,1.149292,-0.481338,1.149292,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840,0.736216,0.629546,0.076840,0.736216,0,0,0
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840,0.852571,0.716170,0.076840,0.852571,1,1,1
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197,1.417729,1.149292,1.193197,1.193197,1,1,1
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019,0.304036,-0.409947,0.635019,0.304036,0,0,0


In [209]:
#give all student with a university rating of 4 or higher a 10 point boost on their GRE score
admissions['gre_score_new'] =[gre+10 if uni_rating>=4 else gre for uni_rating,gre in zip(admissions['university_rating'],admissions['gre_score']) ]



In [212]:
#Let's check:
admissions[['gre_score','university_rating', 'gre_score_new']]

,gre_score,university_rating,gre_score_new
0,337,4,347
1,316,3,316
2,322,3,322
3,314,2,314
4,330,5,340
...,...,...,...
380,324,3,324
381,325,3,325
382,330,4,340
383,312,3,312


In [213]:
#split the column into 4 bins using the cut function. Assign this new score to the variable adjusted_gre.
labels = ["Very Low", "Low", "High", "Very High"]
admissions['adjusted_gre']=pd.cut(admissions['gre_score_new'],4, labels = labels) # - bins have the same size

In [215]:
#Let's check:
admissions[['gre_score','university_rating', 'gre_score_new','adjusted_gre']]

,gre_score,university_rating,gre_score_new,adjusted_gre
0,337,4,347,Very High
1,316,3,316,Low
2,322,3,322,High
3,314,2,314,Low
4,330,5,340,Very High
...,...,...,...,...
380,324,3,324,High
381,325,3,325,High
382,330,4,340,Very High
383,312,3,312,Low
